# HUQ data processing

In [17]:
import petl as etl
from utils.paths import HUQ, SITE

In [18]:
ticketing = (
    etl
    .fromcsv(HUQ / 'mastercard.csv')
    .convertnumbers()
    .convert('datestamp', etl.dateparser('%Y-%m-%d'))
)

In [19]:
OUTPUT = SITE / 'experiments/huq/_data'
OUTPUT.mkdir(exist_ok=True, parents=True)

In [20]:
(
    ticketing
    .selecteq('variable', 'average_account_count')
    .cutout('variable')
    .tocsv(OUTPUT / 'footfall_proxy.csv')
)

In [21]:
(
    ticketing
    .selecteq('variable', 'transaction_amount')
    .cutout('variable')
    .tocsv(OUTPUT / 'spend_index.csv')
)